### Including Main Libraries

In [1]:
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt

## Importing Dataset

In [2]:
dataset = pd.read_csv("train.csv")

In [3]:
dataset = dataset.drop('PassengerId', axis=1)
dataset.head(7)

Survived  Pclass                                               Name  \
0         0       3                            Braund, Mr. Owen Harris   
1         1       1  Cumings, Mrs. John Bradley (Florence Briggs Th...   
2         1       3                             Heikkinen, Miss. Laina   
3         1       1       Futrelle, Mrs. Jacques Heath (Lily May Peel)   
4         0       3                           Allen, Mr. William Henry   
5         0       3                                   Moran, Mr. James   
6         0       1                            McCarthy, Mr. Timothy J   

      Sex   Age  SibSp  Parch            Ticket     Fare Cabin Embarked  
0    male  22.0      1      0         A/5 21171   7.2500   NaN        S  
1  female  38.0      1      0          PC 17599  71.2833   C85        C  
2  female  26.0      0      0  STON/O2. 3101282   7.9250   NaN        S  
3  female  35.0      1      0            113803  53.1000  C123        S  
4    male  35.0      0      0            373450   8.0500   NaN        S  
5    male   NaN      0      0            330877   8.4583   NaN        Q  
6    male  54.0      0      0             17463  51.8625   E46        S

In [4]:
dataset.describe()

Survived      Pclass         Age       SibSp       Parch        Fare
count  891.000000  891.000000  714.000000  891.000000  891.000000  891.000000
mean     0.383838    2.308642   29.699118    0.523008    0.381594   32.204208
std      0.486592    0.836071   14.526497    1.102743    0.806057   49.693429
min      0.000000    1.000000    0.420000    0.000000    0.000000    0.000000
25%      0.000000    2.000000   20.125000    0.000000    0.000000    7.910400
50%      0.000000    3.000000   28.000000    0.000000    0.000000   14.454200
75%      1.000000    3.000000   38.000000    1.000000    0.000000   31.000000
max      1.000000    3.000000   80.000000    8.000000    6.000000  512.329200

### Removing the missing data

In [5]:
null_columns=dataset.columns[dataset.isnull().any()]
dataset[null_columns].isnull().sum()

Age         177
Cabin       687
Embarked      2
dtype: int64

In [6]:
dataset[dataset["Embarked"].isnull()]


Survived  Pclass                                       Name     Sex  \
61          1       1                        Icard, Miss. Amelie  female   
829         1       1  Stone, Mrs. George Nelson (Martha Evelyn)  female   

      Age  SibSp  Parch  Ticket  Fare Cabin Embarked  
61   38.0      0      0  113572  80.0   B28      NaN  
829  62.0      0      0  113572  80.0   B28      NaN

Since "Embarked" has only 2 missing values, it wouldn't hurt to replace the values by preceeding/following entries

In [7]:
dataset["Embarked"].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [8]:
dataset["Embarked"] = dataset["Embarked"].fillna(method='bfill')
dataset[null_columns].isnull().sum()

Age         177
Cabin       687
Embarked      0
dtype: int64

Now that that's done, we see that there are too many missing values of "Cabin" to be imputed. What we can do is make it a binary feature, where all instances of a person having a cabin is replaced by 1 and of not having a cabin listed by 0.<br>
0 : Does not have a cabin listed<br>
1 : Has a Cabin listed<br>

In [9]:
cabins = dataset['Cabin'].values

dataset.loc[~dataset["Cabin"].isnull(),"Cabin"] = 1
dataset.loc[dataset["Cabin"].isnull(),"Cabin"] = 0


In [10]:
dataset.head(10)

Survived  Pclass                                               Name  \
0         0       3                            Braund, Mr. Owen Harris   
1         1       1  Cumings, Mrs. John Bradley (Florence Briggs Th...   
2         1       3                             Heikkinen, Miss. Laina   
3         1       1       Futrelle, Mrs. Jacques Heath (Lily May Peel)   
4         0       3                           Allen, Mr. William Henry   
5         0       3                                   Moran, Mr. James   
6         0       1                            McCarthy, Mr. Timothy J   
7         0       3                     Palsson, Master. Gosta Leonard   
8         1       3  Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)   
9         1       2                Nasser, Mrs. Nicholas (Adele Achem)   

      Sex   Age  SibSp  Parch            Ticket     Fare  Cabin Embarked  
0    male  22.0      1      0         A/5 21171   7.2500      0        S  
1  female  38.0      1      0          PC 17599  71.2833      1        C  
2  female  26.0      0      0  STON/O2. 3101282   7.9250      0        S  
3  female  35.0      1      0            113803  53.1000      1        S  
4    male  35.0      0      0            373450   8.0500      0        S  
5    male   NaN      0      0            330877   8.4583      0        Q  
6    male  54.0      0      0             17463  51.8625      1        S  
7    male   2.0      3      1            349909  21.0750      0        S  
8  female  27.0      0      2            347742  11.1333      0        S  
9  female  14.0      1      0            237736  30.0708      0        C

In [11]:
dataset[null_columns].isnull().sum()

Age         177
Cabin         0
Embarked      0
dtype: int64

When it comes to age, the problem is that there are enough missing values that make regular replacement an issue for the predictive model (Model may get wrongly biased).<br>
Now either we replace the values of age with some other value based on a linear regression model that is based on all other columns.<br>
I'm keen on trying multiple Imputation to solve this issue as multiple articles have referenced its relatively low bias on the model. However, to apply such a imputation model, I would have to first further clean the data

One more step that should ideally come under the Data Preprocessing phase is the seperation of "Title" (Mr, Mrs, Master etc) from name. Since all names seem to have a title, it wouldn't hurt to test the statistical importance of the respective titles

In [12]:
titles = []
for name in dataset['Name']:
    fname = name.split(',')
    title = fname[1].split('.')[0]
    titles.append(title)
dataset['Title'] = pd.Series(titles)
dataset = dataset.drop('Name',axis = 1)


In [13]:
ttype = []
for ticket in dataset['Ticket']:
    typetick = ticket.split()[0]
    try:
        int(typetick)
        ttype.append('Regular')
    except:
        ttype.append(typetick)

dataset['TicketType'] = pd.Series(ttype)
dataset.drop('Ticket',axis = 1)
dataset['TicketType'].describe()

count         891
unique         44
top       Regular
freq          661
Name: TicketType, dtype: object

In [34]:
dataset["TicketType"].value_counts()


Regular       661
PC             60
C.A.           27
A/5            21
STON/O         12
W./C.           9
CA.             8
SOTON/O.Q.      8
A/4             7
SOTON/OQ        7
STON/O2.        6
CA              6
C               5
SC/PARIS        5
S.O.C.          5
F.C.C.          5
LINE            4
SC/Paris        4
SC/AH           3
S.O./P.P.       3
PP              3
S.C./PARIS      2
P/PP            2
WE/P            2
SOTON/O2        2
S.P.            1
SO/C            1
SC              1
C.A./SOTON      1
F.C.            1
W.E.P.          1
SCO/W           1
S.W./PP         1
W/C             1
Fa              1
S.O.P.          1
A/S             1
SW/PP           1
S.C./A.4.       1
Name: TicketType, dtype: int64

Clearly, there is a repition of elements that may mean the same thing.<br>
For example, A.5., A.5, A.5/<br>
This is something that requires correction.<br>

In [47]:
dataset['TicketType'].replace(['A/5','A/5.','A.5.','A./5.'],'A/5',inplace = True)
dataset['TicketType'].replace(['A/4','A/4.','A4.'],'A/4',inplace = True)
dataset['TicketType'].replace(['C.A.','C.A./SOTON','CA','CA.'],'CA',inplace = True)
dataset['TicketType'].replace(['S.C./A.4.','S.C./PARIS','SC','SC/AH','SC/PARIS','SC/Paris'],'SC',inplace = True)
dataset['TicketType'].replace(['STON/O2.','SOTON/O.Q.'],['SOTON/02','SOTON/OQ'],inplace = True)
dataset['TicketType'].replace(['W./C.','W/C'],'W/C',inplace = True)
dataset['TicketType'].replace('W.E.P','WE/P',inplace = True)
dataset['TicketType'].replace(['SOTON/OQ','SOTON/O2','SOTON/02'],'STON/O',inplace = True)

normal = ['Regular','PC','CA','STON/O','A/5','SC','W/C','Other']
i = 0
for row in dataset['TicketType']:
       if row not in normal:
            dataset['TicketType'].replace(row,'Other',inplace = True)
            i = i+1
print(i)
dataset["TicketType"].value_counts()

19


Regular    661
PC          60
Other       46
CA          42
STON/O      35
A/5         21
SC          16
W/C         10
Name: TicketType, dtype: int64

# Data Analysis

In [ ]:
sb.histogram()

# Data Pre-Processing

In [33]:
dataset.head(10)

Survived  Pclass     Sex   Age  SibSp  Parch            Ticket     Fare  \
0         0       3    male  22.0      1      0         A/5 21171   7.2500   
1         1       1  female  38.0      1      0          PC 17599  71.2833   
2         1       3  female  26.0      0      0  STON/O2. 3101282   7.9250   
3         1       1  female  35.0      1      0            113803  53.1000   
4         0       3    male  35.0      0      0            373450   8.0500   
5         0       3    male   NaN      0      0            330877   8.4583   
6         0       1    male  54.0      0      0             17463  51.8625   
7         0       3    male   2.0      3      1            349909  21.0750   
8         1       3  female  27.0      0      2            347742  11.1333   
9         1       2  female  14.0      1      0            237736  30.0708   

   Cabin Embarked    Title TicketType  
0      0        S       Mr        A/5  
1      1        C      Mrs         PC  
2      0        S     Miss   STON/O2.  
3      1        S      Mrs    Regular  
4      0        S       Mr    Regular  
5      0        Q       Mr    Regular  
6      1        S       Mr    Regular  
7      0        S   Master    Regular  
8      0        S      Mrs    Regular  
9      0        C      Mrs    Regular

First things first.<br>
Converting all the string variables (excluding name) to categorical variables is an obvious starting point.

Lets take Sex and convert Male to 0 and Female to 1<br>
Male : 0<br>
Female : 1<br>

In [35]:
pd.get_dummies(dataset,columns = ['Sex','Embarked','Title'],drop_first=True)

Survived  Pclass   Age  SibSp  Parch            Ticket      Fare  Cabin  \
0           0       3  22.0      1      0         A/5 21171    7.2500      0   
1           1       1  38.0      1      0          PC 17599   71.2833      1   
2           1       3  26.0      0      0  STON/O2. 3101282    7.9250      0   
3           1       1  35.0      1      0            113803   53.1000      1   
4           0       3  35.0      0      0            373450    8.0500      0   
5           0       3   NaN      0      0            330877    8.4583      0   
6           0       1  54.0      0      0             17463   51.8625      1   
7           0       3   2.0      3      1            349909   21.0750      0   
8           1       3  27.0      0      2            347742   11.1333      0   
9           1       2  14.0      1      0            237736   30.0708      0   
10          1       3   4.0      1      1           PP 9549   16.7000      1   
11          1       1  58.0      0      0            113783   26.5500      1   
12          0       3  20.0      0      0         A/5. 2151    8.0500      0   
13          0       3  39.0      1      5            347082   31.2750      0   
14          0       3  14.0      0      0            350406    7.8542      0   
15          1       2  55.0      0      0            248706   16.0000      0   
16          0       3   2.0      4      1            382652   29.1250      0   
17          1       2   NaN      0      0            244373   13.0000      0   
18          0       3  31.0      1      0            345763   18.0000      0   
19          1       3   NaN      0      0              2649    7.2250      0   
20          0       2  35.0      0      0            239865   26.0000      0   
21          1       2  34.0      0      0            248698   13.0000      1   
22          1       3  15.0      0      0            330923    8.0292      0   
23          1       1  28.0      0      0            113788   35.5000      1   
24          0       3   8.0      3      1            349909   21.0750      0   
25          1       3  38.0      1      5            347077   31.3875      0   
26          0       3   NaN      0      0              2631    7.2250      0   
27          0       1  19.0      3      2             19950  263.0000      1   
28          1       3   NaN      0      0            330959    7.8792      0   
29          0       3   NaN      0      0            349216    7.8958      0   
..        ...     ...   ...    ...    ...               ...       ...    ...   
861         0       2  21.0      1      0             28134   11.5000      0   
862         1       1  48.0      0      0             17466   25.9292      1   
863         0       3   NaN      8      2          CA. 2343   69.5500      0   
864         0       2  24.0      0      0            233866   13.0000      0   
865         1       2  42.0      0      0            236852   13.0000      0   
866         1       2  27.0      1      0     SC/PARIS 2149   13.8583      0   
867         0       1  31.0      0      0          PC 17590   50.4958      1   
868         0       3   NaN      0      0            345777    9.5000      0   
869         1       3   4.0      1      1            347742   11.1333      0   
870         0       3  26.0      0      0            349248    7.8958      0   
871         1       1  47.0      1      1             11751   52.5542      1   
872         0       1  33.0      0      0               695    5.0000      1   
873         0       3  47.0      0      0            345765    9.0000      0   
874         1       2  28.0      1      0         P/PP 3381   24.0000      0   
875         1       3  15.0      0      0              2667    7.2250      0   
876         0       3  20.0      0      0              7534    9.8458      0   
877         0       3  19.0      0      0            349212    7.8958      0   
878         0       3   NaN      0      0            349217    7.8958      0   
879  

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Encoding Sex
labelencoder_sex = LabelEncoder()
dataset.loc[:, 'Sex'] = labelencoder_sex.fit_transform(dataset.loc[:, 'Sex'])

# Encoding Embarked
labelencoder_embarked = LabelEncoder()
dataset.loc[:,'Embarked'] = labelencoder_embarked.fit_transform(dataset.loc[:, 'Embarked'])

# Binarizing Embarked because it makes more sense here
#onehotencode_Embarked = OneHotEncoder(categorical_features=[9])
#onehotencode_Embarked.fit_transform(dataset).toarray()
